this is a markdown comment. the ipynb format is useful for getting individual cells to run

In [3]:
x = "Here we go!"
print(x)

Here we go!


In [12]:
import sys
print(sys.version)


3.13.5 (v3.13.5:6cb20a219a8, Jun 11 2025, 12:23:45) [Clang 16.0.0 (clang-1600.0.26.6)]


In [ ]:
# Cell 1 - Imports and Setup
import requests
import pandas as pd

headers = {'User-Agent': 'aditya.goel.tx@gmail.com'}
